# Hypothesis Testing

In this notebook, we are going to represent the logic of our hypothesis testing for comparing NATS config changes. Everytime that we make a change, we benchmark the before and after change cases. After that, we change the groups names based on ```tmp``` directory names. Then we run this notebook in order to check the compare results.

In [57]:
# notebook variables
GROUP_A="ddup-5.csv" # change this
GROUP_B="nddup-5.csv" # change this

# ttest variables
P_VALUE_BOUND = 0.05 # change the pvalue bound
PERMUTATION_VALUE = 25 # you can update permutation value (ususally we set it to half of our dataset capacity)

In [58]:
# import in-use libraries
import pandas as pd
from scipy.stats import ttest_ind

In [59]:
# read csv to create groups datasets
dfA = pd.read_csv(f'../tmp/{GROUP_A}')
dfB = pd.read_csv(f'../tmp/{GROUP_B}')

dfA.head()

,pub-stats,sub-stats,overall-stats
0,6.18,4.31,8.62
1,7.02,4.11,8.21
2,6.29,4.64,9.22
3,5.88,3.72,7.44
4,5.85,3.90,7.81


In [60]:
# describe datasets
dfA.describe()

,pub-stats,sub-stats,overall-stats
count,50.000000,50.000000,50.000000
mean,6.136000,4.132600,8.245600
std,0.730449,0.512703,1.016613
min,4.650000,2.960000,5.800000
25%,5.652500,3.735000,7.477500
50%,6.190000,4.135000,8.255000
75%,6.700000,4.492500,8.977500
max,7.740000,5.130000,10.150000


In [61]:
# using a function for hypothesis testing logic
# which gets two columns data to compare
def hypo_test(groupA, groupB):
    # get mean values
    mean_a = groupA.mean()
    mean_b = groupB.mean()
    
    t_statistic, p_value = ttest_ind(groupA, groupB, equal_var=True, permutations=PERMUTATION_VALUE)
    
    print(f"\tmean of `{GROUP_A}`: {mean_a}")
    print(f"\tmean of `{GROUP_B}`: {mean_b}")
    print(f"\tt-statistic: {t_statistic}")
    print(f"\tp-value: {p_value}")
    
    if p_value < P_VALUE_BOUND:
        print("the difference is statistically significant at 95% confidence level.")
        if mean_a > mean_b:
            print(f"`{GROUP_A}` is better by {100 * float((mean_a-mean_b)/mean_a)}%.")
        else:
            print(f"`{GROUP_B}` is better by {100 * float((mean_b-mean_a)/mean_b)}%.")
    else:
        print("the difference is not statistically significant at 95% confidence level.")

In [62]:
# defining dataset columns
columns = ["pub-stats", "sub-stats", "overall-stats"]

In [63]:
#  hypothesis for comparing two groups
for column in columns:
    print(f"\ntesting `{column}` field:")
    hypo_test(dfA[column], dfB[column])


testing `pub-stats` field:
	mean of `ddup-5.csv`: 6.136
	mean of `nddup-5.csv`: 8.745000000000001
	t-statistic: -12.591037962174664
	p-value: 0.0
the difference is statistically significant at 95% confidence level.
`nddup-5.csv` is better by 29.834190966266444%.

testing `sub-stats` field:
	mean of `ddup-5.csv`: 4.1326
	mean of `nddup-5.csv`: 4.933
	t-statistic: -7.761687469683483
	p-value: 0.0
the difference is statistically significant at 95% confidence level.
`nddup-5.csv` is better by 16.225420636529492%.

testing `overall-stats` field:
	mean of `ddup-5.csv`: 8.245600000000001
	mean of `nddup-5.csv`: 9.866800000000001
	t-statistic: -7.8919422404358075
	p-value: 0.0
the difference is statistically significant at 95% confidence level.
`nddup-5.csv` is better by 16.43085904244537%.
